# 1. Librerías & Set Up

👉🏻 Instalaremos todas las librerias necesarias para poder trabajar con PySpark y Neo4j

In [17]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=8956e8e4e3d5b5618c31f7e8e9b36be54596fe4a6681047e10ec63ff4c7807e2
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [13]:
!pip install neo4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.8/286.8 kB 5.2 MB/s eta 0:00:00


👉🏻 Importaremos los archivos de la base de datos Core para poder trabajar un grafo en Neo4j.

In [102]:
!wget https://www.dropbox.com/scl/fi/2in2ohw1bgkjzeclauevo/neo4j-graph.zip?rlkey=fb2t2ocmj13me0ietrmmy5ij9&st=hi947a3c&dl=0
!mv neo4j-graph.zip?rlkey=fb2t2ocmj13me0ietrmmy5ij9 neo4j-graph.zip
!unzip -qq neo4j-graph.zip
print('* Bases de datos cargada correctamente *')

--2024-06-15 00:32:41--  https://www.dropbox.com/scl/fi/2in2ohw1bgkjzeclauevo/neo4j-graph.zip?rlkey=fb2t2ocmj13me0ietrmmy5ij9
Resolving www.dropbox.com (www.dropbox.com)... 162.125.85.18, 2620:100:6035:18::a27d:5512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.85.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc5fb1414a467c3411ff075cbbae.dl.dropboxusercontent.com/cd/0/inline/CU1N2VjpEn0lUOtmK38dHJm3OWyFVUdJUIttW1mcL8pYvFVNWv96lV93ZLltj5sdt9BKkF3IO5QrVco540CD6vEURIYjqr197bF-0iTHGxwcPDVBVgKaXKsxFWr2g6ixP35Aib-FVRQ78daEDtZErtKD/file# [following]
--2024-06-15 00:32:41--  https://uc5fb1414a467c3411ff075cbbae.dl.dropboxusercontent.com/cd/0/inline/CU1N2VjpEn0lUOtmK38dHJm3OWyFVUdJUIttW1mcL8pYvFVNWv96lV93ZLltj5sdt9BKkF3IO5QrVco540CD6vEURIYjqr197bF-0iTHGxwcPDVBVgKaXKsxFWr2g6ixP35Aib-FVRQ78daEDtZErtKD/file
Resolving uc5fb1414a467c3411ff075cbbae.dl.dropboxusercontent.com (uc5fb1414a467c3411ff075cbbae.dl.dropboxusercontent.com)... 162.125.85.15,

In [103]:
from pyspark.sql import SparkSession
from itertools import product

In [259]:
import neo4j
from neo4j import GraphDatabase
import pandas as pd
import numpy as np

In [165]:
# Iniciamos una sesion en PySpark
spark = SparkSession.builder \
    .getOrCreate()

sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=pyspark-shell>

# 2. Neo4j Graphs and others examples

👉🏻 Las siguientes funciones permiten cargar grafos de ejemplos para poder realizar las consultas de los patrones de grafo.

👉🏻 Existe la funcion create_neo4j_graph para poder cargar una base de datos en Neo4j y luego obtener las aristas existentes de este grafo.

👉🏻 Existe la funcion load_graph para poder seleccionar el grafo que queremos realizar las consultas.

In [277]:
def load_neo4j_graph():
  """
  Funcion para cargar la base de datos de twitter como ejemplo de grafo en Neo4j.
  """
  # Realizamos la conexion con la base de datos del grafo guardado en Neo4j.
  NEO4J_URI = "neo4j+s://demo.neo4jlabs.com"
  NEO4J_USERNAME = "twitter"
  NEO4J_PASSWORD = "twitter"
  AUTH = (NEO4J_USERNAME, NEO4J_PASSWORD)

  driver = GraphDatabase.driver(NEO4J_URI, auth=AUTH)
  with driver.session() as session:
      try:
          session.run("RETURN 1")
          print("Connection to Neo4j established successfully!")
      except Exception as e:
          print(f"Failed to connect to Neo4j: {e}")
      # Query que permite cargar todas las aristas de cada usuario
      # (descartamos el usuario Neo4j para tener una cantidad mas variada xd,
      # ya que si lo dejabamos se repetia mucho)
      query = """
      MATCH (u1:User)-[r:INTERACTS_WITH | SIMILAR_TO]->(u2:User)
      WHERE u1.name <> "Neo4j" AND u2.name <> "Neo4j"
      RETURN u1.name as user1, type(r) AS relationship, u2.name AS user2
      """
      result = session.run(query)
      df_edges = pd.DataFrame(result.data())
      relationship = df_edges['relationship'].unique()

      # Definimos una clase para cada relacion ...
      relationship_class = {i : rel for rel, i in enumerate(relationship)}
      for inter, inter_class in relationship_class.items():
          print(f"{inter} AS class {inter_class}")
      df_edges['relationship'].replace(relationship_class, inplace = True)

      # Transformamos cada usuario a un numero (su identificador) en base
      # a su nombre de usuari que debe ser unico ...
      users1 = df_edges['user1'].unique().tolist()
      users2 = df_edges['user2'].unique().tolist()
      users = list(set(users1 + users2))
      users_id = {user : id for id, user in enumerate(users)}
      df_edges['user1'].replace(users_id, inplace = True)
      df_edges['user2'].replace(users_id, inplace = True)

      np.save('users_id', users_id, allow_pickle = True)
      np.save('relationship_class', relationship_class, allow_pickle = True)

      # Cargamos el grafo en una lista de tuplas
      neo4f_graph = [(item[0], item[1], item[2]) for item in df_edges.values]

  return neo4f_graph


In [252]:
def load_graph(option):
  """
  Funcion que selecciona el grafo de ejemplo, retorna una lista de tuplas
  de las aristas del grafo.
  """
  if option == "neo4j-graph":
    neo4f_graph = load_neo4j_graph()
    return neo4f_graph

  elif option == "basic-example":
    return [
        (1,11,2), (1,11,3), (2,11,3), (3,11,2),
        (3,11,4), (4,11,1), (4,11,2), (4,11,3),
        (4,12,5), (5,12,1), (5,12,2), (5,12,6),
        (1,12,2), (2,12,3), (3,12,1)
    ]
  else:
    print("La opcion no existe :(")
    return None

# 3. MapReduce Algorithm Implementation

👉🏻 Las siguientes funciones permiten desarrollar el algoritmo MapReduce implementado con PySpark

In [80]:
def hash(n):
  """
  Returns number mod 2. The ouput will be 0 or 1.
  """
  return n % 2

👉🏻 Las funciones siguientes permiten simular la fase de mapeo del algoritmo.

In [81]:
def get_keys(edge, b_dim, b_set, pattern_dim):
  """
  output: retorna las llaves correspondientes para un vertice.

  Idea general: buscamos el par hash_n1,hash_n2 dentro de las posibles
  combinaciones dentro del espacio de imagenes de la funcion de hash.
  Dentro del for, obtenemos un string con la codificacion de las llaves y luego
  verificamos si es una llave candidata para el vertice entregado:

  1.  sequence_in_reducer: Si la secuencia 'b1b2' esta en la llave del reducer
      codificada como hash(n1)hash(n2) ?, donde ? = 0 o 1, entonces se considerará
      el par reducer_key : edge.
  2.  edge_case: El otro caso, es para cuando tenemos por ejemplo x = n2 y z = n2
      para patrones de 3 vertices.
  """

  hash_n1 = hash(edge[0]) # valor de hash para el nodo 1 = b1
  hash_n2 = hash(edge[2]) # valor de hash para el nodo 2 = b2
  values = [] # posible keys
  sequence = '{}{}'.format(hash_n1, hash_n2)

  for i in range(0, b_dim ** pattern_dim):
    reducer = ''.join(str(num) for num in b_set[i])
    sequence_in_reducer = sequence in reducer
    edge_case = reducer[0] == sequence[1] and reducer[pattern_dim - 1] == sequence[0]
    if sequence_in_reducer or edge_case:
        reducer_key = tuple(int(digit) for digit in reducer)
        values.append((reducer_key, edge))

  return values


def map_phase(rdd, b_dim, b_set, pattern_dim):
  """
  input:
    - rdd: RDD del grafo de dimension 'dim'
    - b_dim: Cantidad de elementos de las imagenes de la funcion de hash.
    - b_set: Imagenes de la funcion de hash.
    - pattern_dim: cantidad de nodos del patron de grafo.
  ouput: Mapeo de cada arista con respecto a las llaves
  """

  mapped_keys = rdd.flatMap(lambda edge: get_keys(edge, b_dim, b_set, pattern_dim))
  reducers = mapped_keys.groupByKey().mapValues(list)
  return reducers

👉🏻 Las siguientes funciones permiten simular la fase de reduce del algoritmo.

In [82]:
def find_patterns(edges, pattern_dim):
    """
    Funcion que retorna todos los patrones encontrados de la forma
    (n1,...,nl) : [[(n1,label,n2), ..., (nl-1,label,nl)], ... ]
    donde (n1,...,nl) corresponde a los nodos que forman el patron
    y la llave corresponde a los patrones posibles formados por estos nodos
    para una etique
    """
    neighbors = get_neighbors(edges)
    cycles = find_cycles(edges, neighbors, pattern_dim)
    return cycles


def get_neighbors(edges):
    """
    Funcion que retorna todas las aristas consecutivas a otras aristas.
    Es decir, aristas vecinas.
    """
    neighbors = []
    for i in range(len(edges)):
        curr_edge = edges[i]
        neighbors.append([])
        for j in range(len(edges)):
            next_edge = edges[j]
            if next_edge == curr_edge:
                continue
            if curr_edge[2] == next_edge[0]:
                neighbors[i].append(next_edge)
    return neighbors


def find_cycles(edges, neighbors, pattern_dim):
    """
    Funcion que verifica si existe algun grafo ciclico en el grafo de una
    dimension pattern_dim. Retorna todos los ciclos encontrados.
    Utilizamos el algoritmo DFS iterativo para la busqueda de ciclos.
    """
    cycles = [] # Lista de ciclos encontrados

    for edge in edges: # Por cada arista del grafo
        visited = set() # Determinamos las aritas que ya han sido visitadas
        # Inicializamos el stack con con la
        stack = [(edge, [edge])]
        # arista donde empezamos el recorrido

        while len(stack):
            curr_edge, path = stack.pop() # Extraemos la primera arista del stack
            # Recorremos esta arista, por lo que la marcamos como visitada
            visited.add(curr_edge)
            # Caso 1: Si el camino recorrido actual supera la cantidad de nodos del bgp,
            #         descartamos este camino
            if len(path) > pattern_dim:
                continue
            # Caso 2: Si el camino recorrido tiene un largo igual a pattern_dim,
            #         y son aristas transitivas, entonces un posible ciclo.
            first_node = path[0][0] # primer nodo del camino
            last_node = path[-1][2] # ultimo nodo del caminmo
            if len(path) == pattern_dim and first_node == last_node:
                cycles.append(path)
                # Agregamos el camino y seguimos recorriendo
                continue
            # Si no es ciclico el camino desde el nodo actual,
            # entonces iteramos dentro de sus vecinos
            for neighbor_edge in neighbors[edges.index(curr_edge)]:
                if neighbor_edge not in visited:
                    stack.append((neighbor_edge, path + [neighbor_edge]))

    return cycles


def get_nodes(pattern, pattern_dim):
    """
    Funcion que retorna la tupla con los nodos que forman un ciclo.
    """
    nodes = []
    for i in range(pattern_dim):
      edge = pattern[i]
      nodes.append(edge[0])

    return (tuple(sorted(nodes)), tuple(sorted(pattern)))


def get_unique_lists(pattern1, pattern2):
    """
    Funcion que elimina patrones de grafo duplicados para cada llave
    (n1,n2, ..., nl). Si tenemos l = 3, entonces seria (n1,n2,n3)
    """
    lst = []
    if pattern2 != pattern1:
      return lst.append(pattern2)
    else:
      return pattern1


def reduce_phase(reducers, pattern_dim):
    """
    input: RDD del grafo y cantidad de nodos del patron de grafo.
    output: patrones encontrados.
    """
    reducers_edges = reducers.map(lambda v: v[1]) \
                              .filter(lambda x: len(x) >= pattern_dim)
    patterns = reducers_edges \
              .map(lambda edges: find_patterns(edges, pattern_dim)) \
              .flatMap(list) \
              .map(lambda pattern: get_nodes(pattern, pattern_dim)) \
              .groupByKey().mapValues(lambda v: list(set(v)))


    return patterns

👉🏻 Funcion final que simula el algoritmo MapReduce. Recibe los parametros correspondientes del grafo; las dimensiones del conjunto de imagenes de la funcion de hash; el conjunto de imagenes de la funcion de hash; la dimension del patron de grafo.

In [94]:
def map_reduce(rdd_graph, b_dim, b_set, pattern_dim):
  """
  Funcion que simula el algoritmo MapReduce, en donde se distribuye el grafo
  en diferentes reducers y luego obtenemos los posibles patrones
  de grafos formados combinando las informacion de todos los reducers.
  """
  # Fase de Map: Obtenemos las llaves de cada reducer y el conjunto de aristas mapeados a estas llaves.
  reducers = map_phase(rdd_graph, b_dim, b_set, pattern_dim)
  # Fase Reduce: Obtenemos todos los posibles patrones de L nodos.
  patterns = reduce_phase(reducers, pattern_dim)
  return patterns

# 3. Aplicacion MapReduce en Ejemplo Basico

👉🏻 En esta seccion aplicaremos el algoritmo MapReduce para un grafo de prueba pequeño (el que fue dado al inicio de la tarea, con algunas modificaciones). Este lo guardamos como 'basic-example'.

In [110]:
graph = load_graph('basic-example')
rdd_graph = sc.parallelize(graph)
rdd_graph.collect()

[(1, 11, 2),
 (1, 11, 3),
 (2, 11, 3),
 (3, 11, 2),
 (3, 11, 4),
 (4, 11, 1),
 (4, 11, 2),
 (4, 11, 3),
 (4, 12, 5),
 (5, 12, 1),
 (5, 12, 2),
 (5, 12, 6),
 (1, 12, 2),
 (2, 12, 3),
 (3, 12, 1)]

## 3.1 MapReduce Algorithm for Triangles

👉🏻 Para encontrar triangulos dentro de un grafo utilizaremos el algoritmo Map Reduce para parametros b_0 = 2 y l = 3.

👉🏻 En este caso, b_0=2 ya que al utilizar la funcion de hash modular `f(x) = x mod 2`, generamos un conjunto de 2 imagenes: {0,1}. En consecuencia, los posibles reducers son:
```markdown
REDUCERS = [(0, 0, 0),
            (0, 0, 1),
            (0, 1, 0),
            (0, 1, 1),
            (1, 0, 0),
            (1, 0, 1),
            (1, 1, 0),
            (1, 1, 1)]
```

👉🏻 Dado que estamos buscando subgrafos de 3 nodos, entonces l_0 = 3.

👉🏻 Las aristas se distribuyen dentro de los reducers, por lo que almacenamos el grafo de manera distribuida.En cada reducer verificamos si las aristas generan triangulos, en el caso que si (formaria un ciclo) se almacenará en una lista y retornaremos los nodos correspondientes.




In [111]:
# Dimension de elementos del conjunto de imagenes de la funcion de hash: |{0,1}|
b_0 = 2
# Dimension del patron de grafo (triangulo para este caso)
l_0 = 3
# Conjunto de imagenes de la funcion de hash
b_set_0 = list(range(b_0))
# Posibles reducers
reducers_0 = list(product(b_set_0, repeat=l_0))

patterns = map_reduce(rdd_graph, b_0, reducers_0, l_0)

patterns.collect()

[((2, 3, 4),
  [((2, 11, 3), (3, 11, 4), (4, 11, 2)),
   ((2, 12, 3), (3, 11, 4), (4, 11, 2))]),
 ((1, 2, 3),
  [((1, 12, 2), (2, 12, 3), (3, 12, 1)),
   ((1, 11, 2), (2, 11, 3), (3, 12, 1)),
   ((1, 11, 2), (2, 12, 3), (3, 12, 1)),
   ((1, 12, 2), (2, 11, 3), (3, 12, 1))]),
 ((1, 3, 4), [((1, 11, 3), (3, 11, 4), (4, 11, 1))])]

👉🏻 Al aplicar el algoritmo mapReduce, podemos notar que existen 3 tripletas de nodos que forman un triangulo. En donde estos nodos permiten formar diferentes patrones de grafos triangulares. En este caso, como tenemos mas de una etiqueta distinta, podemos notar que para una tripleta de nodo, forman mas de un triangulo.
```markdown
  (2,3,4); (1,2,3); (1,3,4)
```

👉🏻 Las siguientes funciones se definiran con el fin de desarrollar consultas sobre estos grafos.


In [160]:
def filter_patterns(pattern, pattern_dim, labels):
  """
  Funcion que retorna los patrones filtrados segun el label proporcionado como
  parametro.
  """
  pattern_labels = [edge[1] for edge in pattern]
  for i in range(pattern_dim + 1):
      # verificamos si coinciden los labels ...
      if pattern_labels == labels:
        return pattern
      # Hacemos un desplazamiento hacia la derecha ...
      pattern_labels = pattern_labels[i:] + pattern_labels[:i]

  return None

def bgp_query(rdd_graph, b_dim, reducers, pattern_dim, query):
  """
  Funcion que retorna todos los patrones de grafos de dimension "pattern_dim"
  que coincidan segun la query entregada.
  """
  patterns = map_reduce(rdd_graph, b_dim, reducers, pattern_dim).values().flatMap(list)
  processed_query = query.replace("(", "").replace(")", "").split(",")
  labels = [int(part.strip()) for part in processed_query if part.strip().isdigit()]

  result = patterns.filter(lambda pattern: filter_patterns(pattern, pattern_dim, labels))
  for res in result.collect():
    print(res)

In [161]:
# Realizamos la siguiente consulta en donde queremos obtener todos los triangulos
# donde la etiqueta es 11, independiente del nodo.
bgp_query(rdd_graph, b_0, reducers_0, l_0, "(x,11,y), (y,11,z), (z,11,w)")

((2, 11, 3), (3, 11, 4), (4, 11, 2))
((1, 11, 3), (3, 11, 4), (4, 11, 1))


## 3.2 MapReduce Algorithm for Squares


👉🏻 Ahora aplicaremos nuestro algoritmo para la busqueda de patrones de grafos de 4 variables. En primer lugar, utilizaremos el mismo algoritmo anterior, pero cambiaremos los parametros de l y b_set. En este caso, utilizaremos l = 4. En consecuencia, obtenemos una cantidad de $2^4$ reducers (definidos como `reducers_1`).

In [135]:
# Como buscaremos los posibles cuadrados a formar, entonces tendremos l=4 nodos
# en el bgp.
l_1 = 4
# Tendremos b^4 reducers posibles.
b_set_1 = list(range(b_0))
reducers_1 = list(product(b_set_1, repeat=l_1))
# aplicamos el algoritmo para patrones de grafos de l=4 nodos, con 2^4 reducers
# y con un conjunto de preimagenes de la funcion de hash {0,1}.
squares_patterns = map_reduce(rdd_graph, b_0, reducers_1, l_1)
squares_patterns.collect() # Obtenemos todos los posibles patrones cuadrados sin importar el label.

[((1, 2, 3, 4),
  [((1, 12, 2), (2, 12, 3), (3, 11, 4), (4, 11, 1)),
   ((1, 12, 2), (2, 11, 3), (3, 11, 4), (4, 11, 1)),
   ((1, 11, 2), (2, 11, 3), (3, 11, 4), (4, 11, 1)),
   ((1, 11, 2), (2, 12, 3), (3, 11, 4), (4, 11, 1))]),
 ((2, 3, 4, 5),
  [((2, 11, 3), (3, 11, 4), (4, 12, 5), (5, 12, 2)),
   ((2, 12, 3), (3, 11, 4), (4, 12, 5), (5, 12, 2))]),
 ((2, 3, 3, 4),
  [((2, 11, 3), (3, 11, 2), (3, 11, 4), (4, 11, 3)),
   ((2, 12, 3), (3, 11, 2), (3, 11, 4), (4, 11, 3))]),
 ((1, 3, 4, 5), [((1, 11, 3), (3, 11, 4), (4, 12, 5), (5, 12, 1))]),
 ((1, 2, 3, 3), [((1, 11, 3), (2, 12, 3), (3, 11, 2), (3, 12, 1))])]

👉🏻 Ahora podemos notar que el algoritmo se demora aproximadamente 1s en correr.

👉🏻 Podemos notar que existen 5 tuplas que forman patrones de grafos cuadrilateros (considerando que tenemos diferentes etiquetas, por lo que algunos nodos se repiten, sin embargo se debe a que presentan labels de aristas distintos).

```markdown
(1,2,3,4), (2,3,4,5), (2,3,3,4), (1,2,4,5), y (1,2,3,3).
```

In [162]:
bgp_query(rdd_graph, b_0, reducers_1, l_1, "(x,11,y), (y,11,z), (z,11,w), (w,11,x)")

((1, 11, 2), (2, 11, 3), (3, 11, 4), (4, 11, 1))
((2, 11, 3), (3, 11, 2), (3, 11, 4), (4, 11, 3))


In [163]:
bgp_query(rdd_graph, b_0, reducers_1, l_1, "(x,11,y), (y,11,z), (z,12,w), (w,12,x)")

((1, 12, 2), (2, 12, 3), (3, 11, 4), (4, 11, 1))
((2, 11, 3), (3, 11, 4), (4, 12, 5), (5, 12, 2))
((1, 11, 3), (3, 11, 4), (4, 12, 5), (5, 12, 1))


# 4. Aplicacion de un grafo a gran escala: Twitter Graph

👉🏻 Ahora cargaremos un grafo de mayor tamaño (700 aristas) para evaluar nuestro algoritmo. El siguiente grafo corresponde a un grafo que modela la interaccion de cada usuario en la red social twitter.
El grafo sera cargado como lista de tuplas, en donde cada tupla modela la relacion entre un usuario y otro. Consideramos las siguientes 2 tipos de relaciones

    Relacion 0: INTERACTS_WITH - El usuario 1 interacciona con otro usuario 2
    Relacion 1: SIMILAR_TO - El usuario 1 tiene gustos similares con el usuario 2

Referencia: https://github.com/neo4j-graph-examples/twitter-v2


In [278]:
graph = load_graph('neo4j-graph')
rdd_graph = sc.parallelize(graph)

Connection to Neo4j established successfully!
INTERACTS_WITH AS class 0
SIMILAR_TO AS class 1


In [279]:
rdd_graph.count() # Cantidad de aristas del grafo

744

## 4.1 MapReduce Algorithm for Triangles

👉🏻 Ahora aplicaremos MapReduce para encontrar los patrones de grafos con forma de triangulo dentro del grafo cargado desde Neo4j.



In [280]:
patterns = map_reduce(rdd_graph, b_0, reducers_0, l_0)
patterns.collect()

[((274, 409, 436), [((274, 1, 436), (409, 1, 274), (436, 0, 409))]),
 ((159, 314, 436), [((159, 0, 314), (314, 1, 436), (436, 0, 159))]),
 ((16, 181, 436), [((16, 0, 181), (181, 1, 436), (436, 0, 16))]),
 ((15, 62, 318),
  [((15, 0, 62), (62, 1, 318), (318, 0, 15)),
   ((15, 1, 318), (62, 1, 15), (318, 0, 62))]),
 ((131, 183, 251), [((131, 0, 251), (183, 0, 131), (251, 1, 183))]),
 ((183, 193, 251), [((183, 0, 193), (193, 1, 251), (251, 1, 183))]),
 ((87, 157, 489),
  [((87, 0, 489), (157, 0, 87), (489, 1, 157)),
   ((87, 1, 157), (157, 0, 489), (489, 1, 87))]),
 ((91, 93, 367), [((91, 1, 93), (93, 0, 367), (367, 1, 91))]),
 ((131, 193, 251),
  [((131, 0, 251), (193, 1, 131), (251, 0, 193)),
   ((131, 0, 193), (193, 1, 251), (251, 1, 131))]),
 ((131, 349, 395), [((131, 0, 395), (349, 1, 131), (395, 1, 349))]),
 ((131, 193, 395), [((131, 0, 193), (193, 0, 395), (395, 1, 131))]),
 ((131, 183, 193),
  [((131, 1, 183), (183, 0, 193), (193, 1, 131)),
   ((131, 0, 193), (183, 0, 131), (193, 

👉🏻 El algoritmo es bien rapido de ejecutar, y podemos ver claramente las relaciones entre diferentes usuarios, para diferentes labels.

Analicemos el siguiente caso:

    (
      (317, 349, 445),
      [
      ((317, 1, 349), (349, 0, 445), (445, 0, 317)),
      ((317, 1, 445), (349, 0, 317), (445, 1, 349))
      ]
    )

In [308]:
users_id = np.load('users_id.npy', allow_pickle=True).item()
dict_users_id = {value : key for key, value in users_id.items()}
relationship_class = np.load('relationship_class.npy', allow_pickle=True).item()

In [321]:
user1 = dict_users_id[317]
user2 = dict_users_id[349]
user3 = dict_users_id[445]
print(f'Usuario 1: {user1}\nUsuario 2: {user2}\nUsuario 3: {user3}')

Usuario 1: Wen.S
Usuario 2: Born To Be Idle
Usuario 3: Brana Rakic


In [323]:
bgp_query(rdd_graph, b_0, reducers_0, l_0, "(x,1,y), (y,0,z), (z,0,w)")

((159, 0, 314), (314, 1, 436), (436, 0, 159))
((16, 0, 181), (181, 1, 436), (436, 0, 16))
((15, 0, 62), (62, 1, 318), (318, 0, 15))
((131, 0, 251), (193, 1, 131), (251, 0, 193))
((317, 1, 349), (345, 0, 317), (349, 0, 345))
((317, 1, 395), (345, 0, 317), (395, 0, 345))
((193, 0, 395), (345, 1, 193), (395, 0, 345))
((131, 0, 395), (345, 1, 131), (395, 0, 345))
((317, 1, 395), (395, 0, 445), (445, 0, 317))
((193, 0, 395), (317, 1, 193), (395, 0, 317))
((131, 0, 395), (317, 1, 131), (395, 0, 317))
((317, 1, 349), (349, 0, 445), (445, 0, 317))
((322, 0, 431), (337, 1, 322), (431, 0, 337))
((71, 1, 189), (189, 0, 232), (232, 0, 71))
((50, 1, 131), (131, 0, 193), (193, 0, 50))
((349, 0, 445), (412, 1, 349), (445, 0, 412))


## 4.2 MapReduce For Squares

In [248]:
patterns = map_reduce(rdd_graph, b_0, reducers_1, l_1)
patterns.collect()

[((157, 183, 437, 437),
  [((157, 0, 437), (183, 0, 437), (437, 2, 157), (437, 2, 183))]),
 ((157, 157, 244, 490),
  [((157, 0, 244), (157, 0, 490), (244, 2, 157), (490, 2, 157))]),
 ((157, 157, 410, 490),
  [((157, 0, 410), (157, 0, 490), (410, 2, 157), (490, 2, 157))]),
 ((157, 275, 410, 410),
  [((157, 0, 410), (275, 0, 410), (410, 2, 157), (410, 2, 275))]),
 ((157, 244, 244, 437),
  [((157, 0, 244), (244, 2, 157), (244, 2, 437), (437, 0, 244))]),
 ((9, 87, 157, 157), [((9, 2, 157), (87, 2, 157), (157, 0, 9), (157, 0, 87))]),
 ((70, 183, 183, 208),
  [((70, 2, 183), (183, 0, 70), (183, 0, 208), (208, 2, 183))]),
 ((131, 183, 183, 193),
  [((131, 2, 183), (183, 0, 131), (183, 0, 193), (193, 2, 183))]),
 ((131, 183, 183, 327),
  [((131, 2, 183), (183, 0, 131), (183, 0, 327), (327, 2, 183))]),
 ((183, 183, 193, 327),
  [((183, 0, 193), (183, 0, 327), (193, 2, 183), (327, 2, 183))]),
 ((183, 183, 193, 437),
  [((183, 0, 193), (183, 0, 437), (193, 2, 183), (437, 2, 183))]),
 ((183, 183, 

In [241]:
bgp_query(rdd_graph, b_0, reducers_1, l_1, "(x,0,y), (y,0,z), (z,1,w), (w,1,x)")

((16, 1, 436), (314, 1, 436), (436, 0, 16), (436, 0, 314))
((157, 0, 436), (183, 0, 436), (436, 1, 157), (436, 1, 183))
((9, 1, 157), (157, 0, 9), (157, 0, 489), (489, 1, 157))
((87, 1, 157), (157, 0, 87), (157, 0, 489), (489, 1, 157))
((157, 0, 243), (157, 0, 489), (243, 1, 157), (489, 1, 157))
((157, 0, 409), (157, 0, 489), (409, 1, 157), (489, 1, 157))
((9, 1, 157), (87, 1, 157), (157, 0, 9), (157, 0, 87))
((87, 1, 157), (157, 0, 87), (157, 0, 243), (243, 1, 157))
((87, 1, 157), (157, 0, 87), (157, 0, 409), (409, 1, 157))
((9, 1, 157), (157, 0, 9), (157, 0, 243), (243, 1, 157))
((9, 1, 157), (157, 0, 9), (157, 0, 409), (409, 1, 157))
((70, 1, 183), (183, 0, 70), (183, 0, 208), (208, 1, 183))
((183, 0, 208), (183, 0, 326), (208, 1, 183), (326, 1, 183))
((131, 1, 183), (183, 0, 131), (183, 0, 193), (193, 1, 183))
((70, 1, 183), (183, 0, 70), (183, 0, 326), (326, 1, 183))
((70, 1, 183), (183, 0, 70), (183, 0, 436), (436, 1, 183))
((183, 0, 326), (183, 0, 436), (326, 1, 183), (436, 1, 1

In [243]:
bgp_query(rdd_graph, b_0, reducers_1, l_1, "(x,0,y), (y,1,z), (z,1,w), (w,1,x)")

((131, 1, 183), (183, 0, 193), (193, 1, 251), (251, 1, 131))
((345, 1, 395), (349, 0, 445), (395, 1, 349), (445, 1, 345))
((157, 0, 243), (183, 1, 157), (243, 1, 436), (436, 1, 183))
((131, 1, 347), (183, 0, 131), (347, 1, 436), (436, 1, 183))
((87, 1, 436), (157, 0, 87), (183, 1, 157), (436, 1, 183))
((157, 0, 409), (183, 1, 157), (409, 1, 436), (436, 1, 183))
((131, 0, 412), (317, 1, 395), (395, 1, 131), (412, 1, 317))
((102, 1, 131), (131, 1, 183), (183, 0, 193), (193, 1, 102))
((193, 0, 412), (317, 1, 395), (395, 1, 193), (412, 1, 317))
((193, 0, 412), (317, 1, 349), (349, 1, 193), (412, 1, 317))
((317, 1, 395), (349, 0, 412), (395, 1, 349), (412, 1, 317))
((193, 0, 412), (317, 1, 345), (345, 1, 193), (412, 1, 317))
((193, 0, 412), (317, 1, 445), (412, 1, 317), (445, 1, 193))
